In [5]:
import pandas as pd
import datetime as dt
import teradata
import pyodbc

In [6]:
#Lee el archivo de los proveedores
def leer_archivos():
    
    #Especificar la ulr para agarrar todos los archivos de ahi, de momento no tengo la url de una compartida entonces por el momento no importa
    #url = ""
    nombre_cts = "Copia de CTS_Procesamiento_2025.06.16.xlsx"
    nombre_int = "Copia de Procesamiento_compras_INTENEG_2025-06-16.xlsx"
    nombre_historico = "Historico.xlsx"
    
    url_cts = nombre_cts
    url_int = nombre_int
    #url_his = url + nombre_historico
    
    dtype = {
        'Cuenta' : str,
        'Tarjeta' : str,
        'Autorizacion1' : str
    }

    #Lee los archivos tal cual como vienen en el sharepoint
    df_cts = pd.read_excel(url_cts, dtype=dtype, parse_dates=["Fcompra1"])
    df_int = pd.read_excel(url_int, dtype = dtype, parse_dates=["Fcompra1"])
    
    df_cts["Fcompra1"] = df_cts["Fcompra1"].dt.strftime("%d/%m/%Y")
    df_int["Fcompra1"] = df_int["Fcompra1"].dt.strftime("%d/%m/%Y")
    
    
    #Esto junta los dos df en uno solo pero no se si es necesario tenerlos separados o si si se puede tenerlos juntos
    df_prove = pd.concat([df_cts,df_int])
    
    return df_prove

def quitar_duplicados(df):

    #Crea la llave
    df["Llave"] = df["Cuenta"] + df["Autorizacion1"]

    #Borramos los duplicados
    df = df.drop_duplicates(subset = 'Llave')
    
    return df
    
#def quitar_historicos(df):
    #Eliminamos las que ya se encuentrann en el historico
    #df_prove = df_prove[~df_prove['Llave'].isin(set(df_historico['Llave']))]

def formato_SQL(df,columna):
    
    #Convertir columnas en formato SQL (con comillas y coma) para la consulta SQL ejemplo 123456 va a ser igual a '123456',
    lista_formato_SQL = df[columna].tolist()
    lista_formato_SQL= ', '.join(f"'{tarjeta}'" for tarjeta in lista_formato_SQL)
    
    return lista_formato_SQL
    
    
def teradata_sebas(cuentas):
    query = f"""SELECT  
        substr(NUM_CAR_CARF,4,16) as PAN
        ,NUM_CTA_CTAF as CTA 
        ,VAL_TRANS_COM as IMPORTE
        ,DAT_TRANS_COM as FECHA_TXN
        ,NOM_ESTB_FPRO as NOMBRE_COMERCIO
        ,NUM_REF_COM as REF_TXN
        ,NOM_FANT_CARF as NOMBRE_CLIENTE
        ,substr(X.VAL_DADO_ORIG_LKP,4,5) AS PLAN 
        ,NUM_AUT_COM
        ,DAT_PROX_CORTE_FAT_ATRF AS Prox_Corte
        ,COD_TIPO_TRANSACAO_FPRO

        FROM CARTAO_FINANCIAMENTO_V	

        INNER JOIN CONTA_FINANCIAMENTO_V
        ON ID_CTA_CARF = ID_CTA_CTAF

        LEFT OUTER JOIN COMPRA_V
        ON ID_CAR_COM = ID_CAR_CARF

        INNER JOIN PLANO_CREDITO_V X
        ON ID_PLANO_CRED_COM = X.ID_PLANO_CRED_LKP

        INNER JOIN TIPO_TRANSACAO_V
        ON ID_TIPO_TRANS_COM = ID_TIPO_TRANS_LKP

        LEFT OUTER JOIN FAT_TRANSACAO_PROSA_V
        ON ID_CAR_FPRO = ID_CAR_COM
        AND NUM_REF_FPRO = NUM_REF_COM

        inner join ATRIBUTO_CONTA_FINANCIAMENTO_V
        on ID_CTA_CTAF = ID_CTA_ATRF and current_date between DAT_DE_ATRF and DAT_ATE_ATRF

        WHERE
        NUM_CTA_CTAF IN ({cuentas})
        and DAT_TRANS_COM >= CURRENT_DATE - INTERVAL '60' DAY
        and substr(X.VAL_DADO_ORIG_LKP,4,5) in ('01000','01100','01181','06000')

    
    """
    
    return query

def conexion_a_teradata():
    
    try:
        print('Iniciando sesión en Teradata...')
    
        host: str = '192.168.17.64'  # Dirección IP o nombre del servidor Teradata
        user: str = 'MPROC_GR'       # Usuario de Teradata
        password: str = 'padrao' # Contraseña de Teradata
        driver: str = "Teradata Database ODBC Driver 20.00"  # Controlador ODBC

        udaExec = teradata.UdaExec(appName="PythonApp", version="1.0", logConsole=False)

        session = udaExec.connect(

            method="odbc",
            system=host,
            username=user,
            password=password,
            driver=driver,
            sslmode='Allow'
        )
        
    except Exception as e:
        print(f"Error en la conexión a Teradata: {str(e)}")
        return None  # Retorna None en caso de error
    
    return session

    
def hacer_consulta(session, query):
    
    session.execute("DATABASE PM_MstrDB")
    df = pd.read_sql(query, session)
    
    return df

def cerrar_conexion(session):
    session.close()

    
def asignar_plan(df, df_teradata):
    
    #Crea la llave corta en teradata cuenta&aut
    df_teradata["Llave"] = df_teradata["CTA"] + df_teradata["NUM_AUT_COM"]
    
    #Busca en la consulta teradata la llave completa y le asigna el plan
    df = pd.merge(df,df_teradata[["Llave","PLAN"]], on="Llave", how='left')
    
    return df
    
    
def correccion_datos(df_mal, df_teradata):
    
    #Crea la llave corta en teradata cuenta&aut
    df_teradata["Llave"] = df_teradata["CTA"] + df_teradata["NUM_AUT_COM"]
    
    df_no_estan = df_mal[~df_mal['Llave'].isin(set(df_teradata['Llave']))]
    df_mal = df_mal[df_mal['Llave'].isin(set(df_teradata['Llave']))]
    
    
    df_mal = pd.merge(df_mal,df_teradata[["Llave","IMPORTE"]], on="Llave", how='left')
    df_mal = pd.merge(df_mal,df_teradata[["Llave","FECHA_TXN"]], on="Llave", how='left')
    
    return df_mal,df_no_estan
    
    
    
def proceso(df):
    #Esta funcion servira por si tenemos que mantener los df separados entonces pasamos los df por esta funcion
    #y ya no escribir cada paso dos veces
    
    df = quitar_duplicados(df)
    
    #Convierte todas las cuentas en formato SQL
    cuentas = formato_SQL(df, "Cuenta")
    
    #Hace la consulta en teradata
    session = conexion_a_teradata()
    df_teradata = hacer_consulta(session, teradata_sebas(cuentas))
    cerrar_conexion(session)
   

    #Cambiar el formato de la fecha a dd/mm/aaaa
    df_teradata['FECHA_TXN'] = pd.to_datetime(df_teradata['FECHA_TXN'], dayfirst=True, errors='coerce')
    df_teradata['FECHA_TXN'] = df_teradata['FECHA_TXN'].dt.strftime('%d/%m/%Y')
    df_teradata['IMPORTE'] = df_teradata['IMPORTE'].map(lambda x: ('%g' % x))
    
    #Quita los 0 que no nos sirven
    df['Monto1'] = df['Monto1'].map(lambda x: ('%g' % x))
    
    #Crear una llave en terada cuenta&aut&monto&fecha?
    df_teradata["Llave_completa"] = df_teradata["CTA"] + df_teradata["NUM_AUT_COM"] + df_teradata["IMPORTE"].astype(str) + df_teradata["FECHA_TXN"].astype(str)
    df["Llave_completa"] = df["Cuenta"] + df["Autorizacion1"] + df["Monto1"].astype(str) + df["Fcompra1"].astype(str)
    
    df = asignar_plan(df, df_teradata)
    
    
    #Busca en las dos consultas de teradata si se encuentran esos casos y asi saber que todos los campos de la llave estan bien
    df_bien = df[df['Llave_completa'].isin(set(df_teradata['Llave_completa']))]
    
    #Los que no tienen algun campo bien, averiguar que campo esta mal y poner el correcto?
    df_mal = df[~df['Llave_completa'].isin(set(df_teradata['Llave_completa']))]
    
    df_mal,df_no_estan = correccion_datos(df_mal,df_teradata)
    
    url = "C:\\Users\\luis.campos\\Downloads\\Parcializaciones\\casos.xlsx"
    
    with pd.ExcelWriter(url) as writer:
        df.to_excel(writer, "Completo", index=False)
        df_bien.to_excel(writer, "Casos Correctos", index=False)
        df_mal.to_excel(writer, "Casos con errores", index=False)
        df_no_estan.to_excel(writer,"No estan en Teradata", index=False)
        
    print("Archivo guardado con exito")
    
    
    return df_bien, df_mal
    
    
    

def main():
    
    
    #Para no hacer todo dos veces en caso de qu los df se deban mantener separados, se puede crear una funcion en donde se ponga 
    #todos los pasos y de parametro reciba un df, finalmente en el main llamamos a esa funcio dos veces una por cada df
    df_prove = leer_archivos()
    df_bien, df_mal = proceso(df_prove)
    
    #print(df_bien)
    
    
    
    
main()



Iniciando sesión en Teradata...


C:\Users\luis.campos\AppData\Local\Temp\ipykernel_6332\1483155452.py:133: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, session)
C:\Users\luis.campos\AppData\Local\Temp\ipykernel_6332\1483155452.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Monto1'] = df['Monto1'].map(lambda x: ('%g' % x))
C:\Users\luis.campos\AppData\Local\Temp\ipykernel_6332\1483155452.py:193: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

Archivo guardado con exito
